In [ ]:
#DON'T RENAME ME - I'M CHANGED EXTERNALLY
MSNAME = "msdir/caracal-1477074305-crosshand_cal.avg.ms"
UNPOL_SOURCE = "PKS1934-638"

In [ ]:
from IPython.display import HTML
HTML('code_toggle.html')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from pyrap.tables import table as tbl
from pyrap.tables import taql as taql
from scipy.stats import kurtosis, skew
%matplotlib inline

In [ ]:
with tbl("%s::FIELD" % MSNAME) as t:
    fnames = list(t.getcol("NAME"))
    print fnames
with tbl("%s::SPECTRAL_WINDOW" % MSNAME) as t:
    chans = t.getcell("CHAN_FREQ", 0).size
    chan_freqs = t.getcell("CHAN_FREQ", 0).flatten() / 1e6
with tbl("%s" % MSNAME) as t:
    nrow = t.nrows()

with tbl("%s" % MSNAME, ack=False) as t:
    upol_source_id = fnames.index(UNPOL_SOURCE)
    with taql('select from $t where FIELD_ID == $upol_source_id') as t1:
        a1 = t1.getcol("ANTENNA1")
        a2 = t1.getcol("ANTENNA2")
        flgs = t1.getcol("FLAG")
        data = t1.getcol("CORRECTED_DATA")

# Stokes parameters and spread

In [ ]:
fsel = np.sum(flgs, axis=2) > 0
I = 0.5 * (data[:,:,0].real + data[:,:,3].real)  
Q = 0.5 * (data[:,:,0].real - data[:,:,3].real)   
U = 0.5 * (data[:,:,1].real + data[:,:,2].real)  
V = 0.5 * (data[:,:,1].imag - data[:,:,2].imag) 
# flag data
I[fsel] = np.nan
Q[fsel] = np.nan
U[fsel] = np.nan
V[fsel] = np.nan

In [ ]:
lin_leakage = np.abs((Q**2 + U**2) / I**2)
total_leakage = np.abs((Q**2 + U**2 + V**2) / I**2)

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(12,4))
ax1.scatter(V[:,:],U[:,:])
ax1.set_xlabel("Stokes V")
ax1.set_ylabel("Stokes U")
ax2.scatter(V[:,:],Q[:,:])
ax2.set_xlabel("Stokes V")
ax2.set_ylabel("Stokes Q")
ax3.scatter(U[:,:],Q[:,:])
ax3.set_xlabel("Stokes U")
ax3.set_ylabel("Stokes Q")
plt.show()

In [ ]:
print "Spread in V:", np.nanmean(np.abs(V[:,:])), "+/-", np.nanstd(np.abs(V[:,:])), "kurt:", kurtosis(np.abs(V.ravel()), nan_policy='omit'), "skew:", skew(np.abs(V[np.logical_not(np.isnan(V))]).ravel()), "Jy"
print "Spread in U:", np.nanmean(np.abs(U[:,:])), "+/-", np.nanstd(np.abs(U[:,:])), "kurt:", kurtosis(np.abs(U.ravel()), nan_policy='omit'), "skew:", skew(np.abs(U[np.logical_not(np.isnan(U))]).ravel()), "Jy"
print "Spread in Q:", np.nanmean(np.abs(Q[:,:])), "+/-", np.nanstd(np.abs(Q[:,:])), "kurt:", kurtosis(np.abs(Q.ravel()), nan_policy='omit'), "skew:", skew(np.abs(Q[np.logical_not(np.isnan(Q))]).ravel()), "Jy"

In [ ]:
lin_leakage[lin_leakage == 0] = np.nan
total_leakage[lin_leakage == 0] = np.nan
print "Mean linear leakage:", np.nanmean(lin_leakage), \
      "+/-", np.nanstd(lin_leakage), \
      "(", 10*np.log10(np.nanmean(lin_leakage) + np.nanstd(lin_leakage)), "~", \
      10*np.log10(np.nanmean(lin_leakage)), "dB)"
print "Mean total leakage:", np.nanmean(total_leakage), \
      "+/-", np.nanstd(total_leakage), \
      "(", 10*np.log10(np.nanmean(total_leakage) + np.nanstd(total_leakage)), "~", \
      10*np.log10(np.nanmean(total_leakage)), "dB)"

In [ ]:
plt.figure(figsize=(12,6))
f = chan_freqs
plt.errorbar(f,
             np.nanmean(Q, axis=0),
             yerr=np.nanstd(Q, axis=0), fmt='o', ecolor='r', capsize=4)
plt.plot(f, np.nanmedian(Q, axis=0), 'b--')
plt.xlabel("Frequency (MHz)")
plt.ylabel("Stokes Q")

plt.show()
plt.figure(figsize=(12,6))
f = chan_freqs
plt.errorbar(f,
             np.nanmean(U, axis=0),
             yerr=np.nanstd(U, axis=0), fmt='o', ecolor='r', capsize=4)
plt.plot(f, np.nanmedian(U, axis=0), 'b--')
plt.xlabel("Frequency (MHz)")
plt.ylabel("Stokes U")

plt.show()
plt.figure(figsize=(12,6))
f = chan_freqs
plt.errorbar(f,
             np.nanmean(V, axis=0),
             yerr=np.nanstd(V, axis=0), fmt='o', ecolor='r', capsize=4)
plt.plot(f, np.nanmedian(V, axis=0), 'b--')
plt.xlabel("Frequency (MHz)")
plt.ylabel("Stokes V (calibration error)")

plt.show()

# Residual leakage

In [ ]:
XY = data[:,:,1]
YX = data[:,:,2]
XX = data[:,:,0]
rel_errXY = np.abs(XY) / np.abs(XX)
rel_errYX = np.abs(YX) / np.abs(XX)
plt.figure(figsize=(12,6))
f = chan_freqs
plt.fill_between(f,
         10*np.log10(np.nanmean(rel_errXY, axis=0)),      
         10*np.log10(np.nanmean(rel_errXY, axis=0) + np.nanstd(rel_errXY, axis=0)),
         label="|XY| / |XX|"
        )
plt.fill_between(f,
         10*np.log10(np.nanmean(rel_errYX, axis=0)),      
         10*np.log10(np.nanmean(rel_errYX, axis=0) + np.nanstd(rel_errYX, axis=0)),      
         label="|YX| / |XX|"
        )
plt.xlabel("Frequency (MHz)")
plt.ylabel("Residual leakage [crosshand power / parallelhand power] (dB)")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
XY = data[:,:,1]
YX = data[:,:,2]
XX = data[:,:,0]
rel_errXY = np.abs(np.imag(XY)) / np.abs(XX)
rel_errYX = np.abs(np.imag(YX)) / np.abs(XX)
plt.figure(figsize=(12,6))
f = chan_freqs
plt.fill_between(f,
         10*np.log10(np.nanmean(rel_errXY, axis=0)),      
         10*np.log10(np.nanmean(rel_errXY, axis=0) + np.nanstd(rel_errXY, axis=0)),
         label="|$\Im$XY| / |XX|"
        )
plt.fill_between(f,
         10*np.log10(np.nanmean(rel_errYX, axis=0)),      
         10*np.log10(np.nanmean(rel_errYX, axis=0) + np.nanstd(rel_errYX, axis=0)),      
         label="|$\Im$YX| / |XX|"
        )
plt.xlabel("Frequency (MHz)")
plt.ylabel("Residual leakage [crosshand imaginary power / parallelhand power] (dB)")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
def baseline_index(a1, a2, no_antennae):
  """
   Computes unique index of a baseline given antenna 1 and antenna 2
   (zero indexed) as input. The arrays may or may not contain
   auto-correlations.
   There is a quadratic series expression relating a1 and a2
   to a unique baseline index(can be found by the double difference
   method)
   Let slow_varying_index be S = min(a1, a2). The goal is to find
   the number of fast varying terms. As the slow
   varying terms increase these get fewer and fewer, because
   we only consider unique baselines and not the conjugate
   baselines)
   B = (-S ^ 2 + 2 * S *  # Ant + S) / 2 + diff between the
   slowest and fastest varying antenna
  :param a1: array of ANTENNA_1 ids
  :param a2: array of ANTENNA_2 ids
  :param no_antennae: number of antennae in the array
  :return: array of baseline ids
  """
  if a1.shape != a2.shape:
    raise ValueError("a1 and a2 must have the same shape!")

  slow_index = np.min(np.array([a1, a2]), axis=0)

  return (slow_index * (-slow_index + (2 * no_antennae + 1))) // 2 + \
         np.abs(a1 - a2)

no_ant = len(set(np.unique(a1)).union(set(np.unique(a2))))
nbl = no_ant * (no_ant - 1) // 2 + no_ant
bli = baseline_index(a1, a2, no_ant)
bl_q3 = np.zeros(nbl)
bl_q2 = np.zeros(nbl)
bl_q1 = np.zeros(nbl)

XY = data[:,:,1]
YX = data[:,:,2]
XX = data[:,:,0]

for b in xrange(nbl):
    XYsel = XY[bli==b]
    XXsel = XX[bli==b]
    rel_err = np.abs(np.abs(XYsel) / np.abs(XXsel))
    bl_q3[b] = np.nanpercentile(rel_err, 75)
    bl_q2[b] = np.nanpercentile(rel_err, 50)
    bl_q1[b] = np.nanpercentile(rel_err, 25)
    
plt.figure(figsize=(12,6))
f = chan_freqs
plt.fill_between(xrange(nbl),
         10*np.log10(bl_q1),      
         10*np.log10(bl_q3),      
        )
plt.plot(xrange(nbl),
         10*np.log10(bl_q2),
         "w--")
plt.xlabel("Baseline index")
plt.ylabel("Residual leakage [crosshand power / parallelhand power] (dB)")
plt.grid(True)
plt.show()

# Quadrature leakage

In [ ]:
plt.figure(figsize=(12,6))
fleak_mean = np.nanmean(lin_leakage, axis=0)
fleak_std = np.nanstd(lin_leakage, axis=0)
f = chan_freqs
plt.fill_between(f,
                 10*np.log10(np.abs(fleak_mean)),
                 10*np.log10(np.abs(fleak_mean + fleak_std)))
plt.xlabel("Frequency (MHz)")
plt.ylabel("$D_{lin}$ (dB)")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
fleak_mean_tot = np.nanmean(total_leakage, axis=0)
fleak_std_tot = np.nanstd(total_leakage, axis=0)
f = chan_freqs
plt.fill_between(f,
                 10*np.log10(np.abs(fleak_mean_tot)),
                 10*np.log10(np.abs(fleak_mean_tot + fleak_std_tot)))
plt.xlabel("Frequency (MHz)")
plt.ylabel("$D_{tot}$ (dB)")
plt.grid(True)
plt.show()